In [1]:
from src.gather_vote_texts import gather_vote_texts

gather_vote_texts()

2025-05-18 15:47:28.500 | INFO     | src.gather_vote_texts:gather_vote_texts:187 - Gathering vote texts from Bundestag website...
2025-05-18 15:47:32.661 | ERROR    | src.gather_vote_texts:gather_vote_texts:198 - Failed to download or process Energiepreisbremse: 'NoneType' object has no attribute 'strip'
2025-05-18 15:47:32.662 | ERROR    | src.gather_vote_texts:gather_vote_texts:199 - Vote data: id                                                20221215_1
name                                      Energiepreisbremse
date                                     2022-12-15 00:00:00
xls_url    https://www.bundestag.de/resource/blob/924716/...
pdf_url    https://www.bundestag.de/resource/blob/926644/...
Name: 112, dtype: object
2025-05-18 15:47:32.868 | ERROR    | src.gather_vote_texts:download_vote_texts:162 - Failed to extract drucksache or date from data/tmp/vote_pdf/20221125_2/drucksachen/7.pdf drucksache: None date: 05.08.2022
2025-05-18 15:47:33.126 | WARNING  | src.gather_vote_texts:ext

In [2]:
import pandas as pd

data = pd.read_parquet("data/parquet/vote_texts.parquet")
data["document_type"].value_counts().head(50)

Beschlussempfehlung und Bericht           489
Gesetzentwurf                             352
Antrag                                    315
Beschlussempfehlung                       224
Unterrichtung                             147
Änderungsantrag                            89
Bericht                                    76
Entschließungsantrag                       45
Ergänzung zu den Beschlussempfehlungen     27
Antwort                                     3
Große Anfrage                               3
Kleine Anfrage                              2
Verordnung                                  1
Name: document_type, dtype: int64

,vote_id,filename,drucksache,date,document_type
1,20250318_3,data/tmp/vote_pdf/20250318_3/drucksachen/1.pdf,20_15096,10.03.2025,Gesetzentwurf


In [3]:
from src.utils import pdf_utils
sample_row = data.sample(1).iloc[0]
pdf_utils.extract_first_page_text(
    sample_row["filename"],
).strip().splitlines(), sample_row["document_type"]

(['Deutscher Bundestag',
  'Drucksache 19/10706',
  '19. Wahlperiode ',
  '05.06.2019',
  'Beschlussempfehlung und Bericht ',
  'des Ausschusses für Inneres und Heimat (4. Ausschuss)  ',
  'zu dem Gesetzentwurf der Bundesregierung ',
  '– Drucksachen 19/10047, 19/10506 – ',
  'Entwurf eines Zweiten Gesetzes zur besseren Durchsetzung der Ausreisepflicht ',
  'A. Problem ',
  'Die Rechtspflicht, Deutschland zu verlassen, wird von einer hohen Zahl vollzieh-',
  'bar Ausreisepflichtiger nicht befolgt. Sofern die Betroffenen innerhalb der ihnen ',
  'gesetzten Frist ihrer vollziehbaren Ausreisepflicht nicht freiwillig nachkommen, ',
  'muss diese im Wege der Abschiebung durchgesetzt werden.  ',
  'Wesentlicher Teil der Migrationspolitik ist die Rückkehr derer, die unter keinem ',
  'rechtlichen Gesichtspunkt ein Bleiberecht in Deutschland haben. ',
  'Im Bereich der Rückkehr ist eine stärkere Durchsetzung des Rechts erforderlich. ',
  'Die Zuführungsquote zu Rückführungsmaßnahmen soll deutl